In [57]:
"""
Collect Data: Capture and label images of our handwriting.
    [ ] Chelsey
    [ ] Shawn
    [ ] Starter data set (MNIST)

Preprocess Data: Resize, normalize, and encode the images and labels.

Build a CNN Model: Use Keras to define a CNN suitable for image classification.
Train the Model: Fit the model on your handwriting dataset.
Evaluate and Predict: Assess model accuracy and use it to predict new handwriting samples.
"""

import cv2
import numpy as np
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import pickle

In [58]:
# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# LabelBinarizer during training
lb = LabelBinarizer()
lb.fit(y_train)  # Fit the LabelBinarizer with the labels

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

with open("label_binarizer.pkl", "wb") as f:
    pickle.dump(lb, f)


In [59]:
# Build a simple neural network model
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),  # Flatten the 2D image into a 1D vector
    layers.Dense(128, activation='relu'),  # First dense layer with 128 neurons and ReLU activation
    layers.Dropout(0.2),                   # Dropout layer to prevent overfitting
    layers.Dense(10, activation='softmax') # Output layer with 10 classes and softmax activation
])


c:\Users\shawn\anaconda3\envs\dev\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [60]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [61]:
# Train the model
model.fit(x_train, y_train, epochs=5, validation_split=0.2)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8421 - loss: 0.5418 - val_accuracy: 0.9551 - val_loss: 0.1604
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9508 - loss: 0.1700 - val_accuracy: 0.9653 - val_loss: 0.1185
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9646 - loss: 0.1187 - val_accuracy: 0.9693 - val_loss: 0.1011
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9720 - loss: 0.0937 - val_accuracy: 0.9723 - val_loss: 0.0937
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9752 - loss: 0.0793 - val_accuracy: 0.9743 - val_loss: 0.0875


In [62]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy:.2f}')


313/313 - 1s - 2ms/step - accuracy: 0.9766 - loss: 0.0776
Test accuracy: 0.98


In [66]:
with open("label_binarizer.pkl", "rb") as f:
    lb = pickle.load(f)

# Define preprocess_image function here
def preprocess_image(image_path, img_size=(28, 28)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, img_size)
    image = image / 255.0  # Normalize the image
    
    # Debugging step: print a portion of the preprocessed image
    print(f"Preprocessed image (partial): {image.flatten()[:10]}")  # Print the first 10 pixel values
    
    return image

# Define predict_handwriting function here
def predict_handwriting(image_path, model, lb, img_size=(28, 28)):
    # Preprocess the image
    image = preprocess_image(image_path, img_size)
    print(f"Image shape before prediction: {image.shape}")  # Debugging step
    image = image.reshape(1, img_size[0], img_size[1], 1)
    
    # Predict the class
    prediction = model.predict(image)
    print(f"Prediction output: {prediction}")  # Debugging step
    predicted_label = lb.inverse_transform(prediction)[0]
    
    return predicted_label

# Example usage with your model
image_path = "Resources/Test Images/number4.png"
predicted_label = predict_handwriting(image_path, model, lb)
print(f"Predicted Label: {predicted_label}")

Preprocessed image (partial): [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Image shape before prediction: (28, 28)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Prediction output: [[2.1623563e-26 3.9906134e-12 6.2794352e-06 3.6717242e-01 3.7656715e-37
  6.3282120e-01 3.8766984e-08 1.0420770e-15 6.3793912e-08 7.3396906e-28]]
Predicted Label: 5
